# <center> __Deep Learning__ <br/> <br/><br/>
# <center> учим машину распознaвать текст

## <center> Атабек Дулат - Machine Learning Engineer в Kolesa Group
<br/><br/>
### <center> Опыт работы: работал в Нацбанке РК и BTSD
<br/><br/>
### <center> Гитхаб с презентацией и кодом обучения сетки : 
###    <center> https://github.com/carnotaur/crnn-tutorial

# <center> __План:__
<br/><br/>
##  1. Быстрый обзор методов распознавания
##  2. __CRNN__ - What it is?
## 3. Попытка в 'простое' обьяснение __CNN__
## 4. Попытка(но хуже) в обьяснение __RNN__
## 5. Back to the CRNN
## 6. Распознаем капчу и показываю модель

### <center> Задача - Optical Character Recognition a.k.a __OCR__
<center> <img src="imgs/ocr_working.gif" width=1800>

## <center> Распознавание текста можно разделить на 2 этапа: 
<br/><br/>
###  <center> - Определение местоположение текста (Text Detection)
###  <center> - Распознавание текста (OCR)
<br/><br/>
<center> <img src="imgs/ocr-types.png" width=1600>

## <center>  Как было раньше:
<br/><br/>
<center> <img src="imgs/oldway.png" width=1000>

# <center> Эра __"Глубокого Обучения"__ поменяло все

## <center> Зоопарк recongition моделей
<br/><br/><br/>
<center> <img src="imgs/recongition_types.png" width=2600>

## <center> Convolutional Recurrent Neural Network (CRNN)
<br/><br/><br/>
<center> <img src="imgs/crnn-archit.png" width=1000>

# <center> __Convolutions__ a.k.a Свертки - Как они работают? 
<br/><br/><br/>
<center> <img src="imgs/img_pixels.png"  width=1500>

## <center> Принцип работы сверток 
<br/><br/><br/>
<center> <img src="imgs/kernels.png" width=1500>

## <center> Принцип работы сверток 
<br/><br/><br/>

<center> <img src="imgs/3D_Convolution_Animation.gif" height=1200 width=1200>

## <center> Свертки способны выучить очень сложные признаки <br/><br/>

<center> <img src="imgs/cnn-feaures.png" width=1800>

# <center> __Recurrent Neural Network (RNN)__  - Вторая часть уравнения

## <center>  __RNN__ - отлично подходят для работы с sequences
<center> <img src="imgs/rnn-works.png">

## <center> Как работает один блок RNN
<br/><br/>
<center> <img src="imgs/one_block.png"  width=1500>

## <center> Каждый блок имеет общую матрицу весов $W$ 
<br/><br/><br/><br/>

<center> <img src='imgs/several blocks.png' width=1500>

# <center> Вернемся к __CRNN__

## <center> Convolutional Recurrent Neural Network (CRNN) 
<br/><br/>
<center> <img src="imgs/crnn-archit.png" width=900>

## <center> Переход от image features к sequences
<center> <img src="imgs/features-seq.png" width=1500>

## <center> Зачем использовать __RNN__ ? <br/><br/>

## 1. Возможность комбинировать информацию с нескольких receptive fields для сохранения __контекста__
## 2. По сравнению с прошлыми решениями RNN могут обучаться с помощью __back-propagation__ и могут обучаться сразу СNN и RNN
## 3. RNN работает на последовательнастях c __различной__ длиной

## <center> __LSTM__ вместо обычного __RNN__
<br/><br/><br/><br/>
<center> <img src='imgs/lstm.png' width=1800>

## <center> __Connectionist Temporal Classification (CTC)__ <br/><br/>

  ### <center> $P(L / Y)$, где
  ### <center> $L$ - label sequence, предсказываемый текст
### <center> $Y$ per-frame предсказания модели

## <center> __СTC__ позволяет тренировать сеть используя __Negative Log Likeilihood Loss (NLLL)__

In [ ]:
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from layers import blockCNN, blockRNN
from utils.utils import strLabelConverter, decode_prediction

In [ ]:
class CRNN(nn.Module):
    def __init__(self, hidden_size: int, 
                 vocab_size: int, 
                 bidirectional: bool = True, 
                 dropout: float = 0.5):
        super(CRNN, self).__init__()
        # layers
        resnet = resnet18(pretrained=True)
        modules = list(resnet.children())[:-3]
        self.resnet = nn.Sequential(*modules)
        self.cn6 = blockCNN(256, 256, kernel_size=3, padding=1)
        # RNN + Linear
        self.linear1 = nn.Linear(1024, 256)
        self.gru1 = blockRNN(256, hidden_size, hidden_size, dropout=dropout, 
                             bidirectional=bidirectional)
        self.gru2 = blockRNN(hidden_size, hidden_size, vocab_size, dropout=dropout,
                             bidirectional=bidirectional)
        self.linear2 = nn.Linear(hidden_size * 2, vocab_size)
        
    def forward(self, batch: torch.Tensor): 
        batch_size = batch.size(0)
        # convolutions
        batch = self.resnet(batch)
        batch = self.cn6(batch, use_relu=True, use_bn=True)
        # make sequences of image features
        batch = batch.permute(0, 3, 1, 2)
        n_channels = batch.size(1)
        batch = batch.view(batch_size, n_channels, -1)
        batch = self.linear1(batch)
        # rnn layers
        batch = self.gru1(batch, add_output=True)
        batch = self.gru2(batch)
        # output
        batch = self.linear2(batch)
        batch = batch.permute(1, 0, 2)
        return batch

In [ ]:
alphabet = string.ascii_lowercase + string.digits #все возможные буквы в капче
label_converter = strLabelConverter(alphabet) #энкодер надписей
print(alphabet)
hidden_size = 256
vocab_size = len(alphabet) + 1 # один дополнительный слот для 'blank' знака
bidirectional = True
PATH = 'models/crnn.pt'

crnn = CRNN(hidden_size=hidden_size, vocab_size=vocab_size, bidirectional=bidirectional)
crnn.load_state_dict(torch.load(PATH))

In [ ]:
crnn

In [ ]:
SAMPLE_PATH = 'data/sample/2b827.png'
# load image 
img_array = np.asarray(Image.open(SAMPLE_PATH).convert('RGB'))
# convert to tensor
img_tensor = torch.from_numpy(img_array / 255.)
img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0).to(torch.float32)

descr_list = ['batch size', 'num of channels', 'height', 'width']
for descr, s in zip(descr_list, img_tensor.size()):
    print('%s : %s' % (descr, s))

In [ ]:
logits = crnn(img_tensor)

In [ ]:
descr_list = ['max seq len', 'batch size', 'vocab size']
for descr, s in zip(descr_list, logits.size()):
    print('%s : %s' % (descr, s))

In [ ]:
pred = decode_prediction(logits, label_converter)
plt.figure(figsize=(15, 20))
plt.title(pred, fontsize=40)
plt.imshow(img_array)
plt.axis('off');

# <center> -  Гитхаб с презентацией и кодом обучения сетки :  
# <center> - https://github.com/carnotaur/crnn-tutorial  
# <center>
# <center> - Ссылка на мой linkedin:
# <center> - www.linkedin.com/in/dulatatabek
# <center>
# <center> - Вопросы по презентации можете посылать:
# <center> - predator2013@protonmail.com
    